# 2024 Project

# NSW Electricity Demand Forecast

## Data Exploration

### Read Data

We use 4 data sources: temperature_nsw.csv (provided by the course), totaldemand_nsw.csv (provided by the course), population_nsw.csv (collected from the [website of Austraian Bureau of Statistics](https://www.abs.gov.au/statistics/people/population/national-state-and-territory-population/dec-2022)) and GDP.csv (collected from the [website of Austraian Bureau of Statistics](https://www.abs.gov.au/statistics/economy/national-accounts/australian-national-accounts-national-income-expenditure-and-product/latest-release#state-and-territory-final-demand)). These datasets encompass crucial variables for our analysis, namely temperature, population, GDP, and energy demand. However, the GDP data reflects the national figure rather than being specific to New South Wales (NSW), as such detailed state-level data was not accessible. To adjust for this, we proceed under the assumption that GDP trends in NSW are representative of the national average, thereby applying the Australia-wide GDP statistics to our NSW-centric forecasting model.


In [1]:
# Import necessary libaraies
import pandas as pd

In [2]:
# Read Temperature file
temperature = pd.read_csv('../data/NSW/temperature_nsw.csv')